In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sn
import matplotlib.pyplot as pyplot
import pymc3 as pm

from pymc3 import traceplot


# Linear Regression to verify implementation
from sklearn.linear_model import LinearRegression

# Scipy for statistics
import scipy
import theano

from arviz.utils import Numba
Numba.disable_numba()
Numba.numba_flag


import arviz as az #pymc3 plot functions use Arviz. Either command works. Ex: az.plot_posterior = pm.plot_posterior

In [ ]:
#Import Data
NYCData = "nyc.csv" 
NYCDf = pd.read_csv(NYCData)

NYCDf.drop(["Unnamed: 0", "X", "Case"], axis = 1, inplace = True)
NYCDf.set_index("Restaurant", inplace = True)
NYCDf["Intercept"] = 1

NYCDf

In [ ]:
pyplot.style.use('fivethirtyeight')
pyplot.plot(NYCDf["Food"],NYCDf["Price"], "bo")
pyplot.xlabel("Food Rating")
pyplot.ylabel("Average Price")
pyplot.title("Food Rating vs Average Price")
pyplot.show()

In [ ]:
x = NYCDf["Food"]
y = NYCDf["Price"]

A = np.vstack([x, np.ones(len(x))]).T

In [ ]:
model, SSE = np.linalg.lstsq(A, y, rcond=None)[:2]

print(f"""The values gathered from our data are {model[1]} for our intercept and {model[0]} for our slope.""")

In [ ]:
pyplot.style.use('fivethirtyeight')
_ =pyplot.plot(x, y, 'o', label='Original data')
_ = pyplot.plot(x, model[0]*x + model[1], 'r--', label='OLS Fitted line')
_ = pyplot.xlabel("Food Rating")
_ = pyplot.ylabel("Average Price")
_ = pyplot.title("Food Rating vs Average Price")
_ = pyplot.legend()
pyplot.show()

# Informed Prior

In [ ]:
with pm.Model() as linear_model_informed:
    # Intercept
    intercept = pm.Normal("Intercept", mu = 0, sd = 5)
    
    # Slope 
    slope = slope1 = pm.Normal("slope", mu = 0, sd = 5)
    
    # Standard deviation
    sigma = pm.HalfNormal("sigma", sd = 10)
    
    # Estimate of mean
    mean = intercept + slope * NYCDf["Food"]
    
    #start = pm.find_MAP()
    
    #step = pm.NUTS()
    
#     trace = pm.sample(
#             1000, step, start, 
#             random_seed=42, progressbar=True
#         )
    
    # Observed values
    Y_obs = pm.Normal('Y_obs', mu = mean, sd = sigma, observed = NYCDf["Price"].values)
    
    # Sampler
    step = pm.NUTS()

    # Posterior distribution
    linear_trace_informed = pm.sample(100, step)

In [ ]:
pm.traceplot(linear_trace_informed, figsize = (12, 12))

In [ ]:
pm.plot_posterior(linear_trace_informed, figsize = (5, 5), kind = "hist")

In [ ]:
pm.plot_posterior_predictive_glm(linear_trace_informed, samples = 100, eval=np.linspace(16,25, 100), linewidth = 1, 
                                 color = "red", alpha = 0.8, label = "Posterior Distribution Fitted Line",
                                lm = lambda x, sample: sample["Intercept"] + sample["slope"] * x);
pyplot.scatter(NYCDf["Food"], y.values, s = 12, alpha = 0.8, c = "blue", label = "Observations")
pyplot.title("Posterior Predictions")
pyplot.xlabel("Food Rating")
pyplot.ylabel("Average Price")
pyplot.legend()

In [ ]:
bayes_prediction = linear_trace_informed["Intercept"] + linear_trace_informed["slope"] * 20

In [ ]:
sn.kdeplot(bayes_prediction, label = 'Posterior Prediction')
pyplot.vlines(x = model[1] + model[0] * 20, ymin = 0, ymax = 0.7, label = 'OLS Prediction', colors = 'red', linestyles='--')
pyplot.legend()
pyplot.xlabel("Food Rating")
pyplot.ylabel("Average Price")
pyplot.title("Posterior Prediction for Food Rating of 20")

# Uninformed

In [ ]:
with pm.Model() as linear_model_uninformed:
    # Intercept
    intercept = pm.Normal("Intercept", mu = 0, sd = 100)
    
    # Slope 
    slope = slope1 = pm.Normal("slope", mu = 0, sd = 100)
    
    # Standard deviation
    sigma = pm.HalfNormal("sigma", sd = 10)
    
    # Estimate of mean
    mean = intercept + slope * NYCDf["Food"]
    
    #start = pm.find_MAP()
    
    step = pm.NUTS()
    
#     trace = pm.sample(
#             1000, step, start, 
#             random_seed=42, progressbar=True
#         )
    
    # Observed values
    Y_obs = pm.Normal('Y_obs', mu = mean, sd = sigma, observed = NYCDf["Price"].values)
    
    # Sampler
    step = pm.NUTS()

    # Posterior distribution
    linear_trace_uninformed = pm.sample(10000, step)

In [ ]:
pm.traceplot(linear_trace_uninformed, figsize = (12, 12))

In [ ]:
pm.plot_posterior(linear_trace_uninformed, figsize = (5, 5), kind = "hist")

In [ ]:
pm.plot_posterior_predictive_glm(linear_trace_uninformed, samples = 100, eval=np.linspace(16,25, 100), linewidth = 1, 
                                 color = "red", alpha = 0.8, label = "Posterior Distribution Fitted Line",
                                lm = lambda x, sample: sample["Intercept"] + sample["slope"] * x);
pyplot.scatter(NYCDf["Food"], y.values, s = 12, alpha = 0.8, c = "blue", label = "Observations")
pyplot.title("Posterior Predictions")
pyplot.xlabel("Food Rating")
pyplot.ylabel("Average Price")
pyplot.legend()

In [ ]:
bayes_prediction = linear_trace_uninformed["Intercept"] + linear_trace_uninformed["slope"] * 20

In [ ]:
sn.kdeplot(bayes_prediction, label = 'Posterior Prediction')
pyplot.vlines(x = model[1] + model[0] * 20, ymin = 0, ymax = 0.7, label = 'OLS Prediction', colors = 'red', linestyles='--')
pyplot.legend()
pyplot.xlabel("Food Rating")
pyplot.ylabel("Average Price")
pyplot.title("Posterior Prediction for Food Rating of 20")